### Setup

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Setup imports
from crewai import Agent, Task, Crew, LLM
import os
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv()

# get default OpenRouter API key
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("Please set the OPENROUTER_API_KEY environment variable.")
else:
    print("Using OpenRouter API key from environment variable.")


Using OpenRouter API key from environment variable.


In [2]:
# Setup LLM configuration
llm = LLM(
    model="openrouter/openai/gpt-5-mini",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# setup chroma DB + Client
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

# Set custom storage location
os.environ["CREWAI_STORAGE_DIR"] = "./chroma_db"

### De-bugging

In [4]:
from crewai.utilities.paths import db_storage_path
import os

# Get the base storage path
storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")

# List all CrewAI storage directories
if os.path.exists(storage_path):
    print("\nStored files and directories:")
    for item in os.listdir(storage_path):
        item_path = os.path.join(storage_path, item)
        if os.path.isdir(item_path):
            print(f"📁 {item}/")
            # Show ChromaDB collections
            if os.path.exists(item_path):
                for subitem in os.listdir(item_path):
                    print(f"   └── {subitem}")
        else:
            print(f"📄 {item}")
else:
    print("No CrewAI storage directory found yet.")

CrewAI storage location: /home/sharath/.local/share/chroma_db

Stored files and directories:
📄 latest_kickoff_task_outputs.db
📄 long_term_memory_storage.db


### Agents

In [5]:
# Research and planning agents
researcher = Agent(
    role="Senior Miata {topic} Researcher",
    goal="Uncover knowledge and developments on {topic} for Mazda Miatas",
    backstory="You're a seasoned Mazda researcher with a knack for uncovering the latest knowledge "
    "and developments on {topic} and Mazda Miatas. Known for your ability to find the most relevant "
    "information and present it in a clear and concise manner.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic} for Mazda Miatas",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} for Mazda Miatas."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

In [6]:
# writing agent
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic} for Mazda Miatas",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} for Mazda Miatas. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [7]:
# validation agent
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Tasks

In [8]:
# planning task
plan_task = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic} and Mazda Miatas.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [9]:
# writing task
write_task = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic} on Mazda Miatas.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [10]:
# editing task
edit_task = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Create and run the crew

In [13]:
# Create the crew with agents and tasks
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan_task, write_task, edit_task],
    verbose=True,
    # memory=True,
)

In [14]:
# run the agents
result = crew.kickoff(inputs={"topic": "Exhaust Systems"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6d7e5555-3e99-41ca-b224-5cbea15ab28c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Exhaust Systems and Mazda Miatas.   │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Comprehensive Content Plan — Exhaust Systems for Mazda Miatas (MX‑5)                                           │
│                                                                                                                 │
│  1) Project Summary / Angle                                                                                     │
│  - Title ideas:                                                                                                 │
│    - Ultimate Guide to Exhaust Systems for Mazda Miata (MX‑5): Sound, Power, and What to Buy                    │
│    - Best Exhausts for Miata (NA/NB/NC/ND): Fitment, Sound, and Legal Considerations                            │
│  - Primary angle: A practical, generation-by-generation buyer’s guide that helps Miata owners choose the right  │
│  exhaust components (headers, mid‑pipes, cat‑backs, mufflers, catalytic converters) based on goals (sound,      │
│  performance, emissions compliance, track use), budget, and installation skill.                                 │
│  - Tone: Expert, friendly, actionable. Mix of technical explanation and consumer guidance with model-specific   │
│  recommendations.                                                                                               │
│                                                                                                                 │
│  2) Priority Trends, Key Players, and Noteworthy News (latest priorities to emphasize)                          │
│  - Trends                                                                                                       │
│    - Generation-specific, bolt‑on systems remain most popular; demand for Miata‑specific parts (Flyin’ Miata,   │
│  Racing Beat) continues strong.                                                                                 │
│    - Growing emphasis on legal/emissions-compliant systems (CARB EO numbers) — purchasers in CA and many        │
│  states require CARB-compliant parts.                                                                           │
│    - Sound engineering focus: brands designing systems that reduce on‑road drone while preserving an            │
│  aggressive tone using resonators/Helmholtz chambers.                                                           │
│    - Lightweight materials and premium finishes (T304 stainless, titanium options) for weight savings and       │
│  corrosion resistance.                                                                                          │
│    - Turbo/forced induction popularity (ND and earlier Miatas with turbo conversions) driving more turbo‑back   │
│  and downpipe options.                                                                                          │
│    - Increased interest in turnkey performance packages (headers + mid‑pipe + cat‑back) with matched tuning to  │
│  avoid CELs and maximize gains.                                                                                 │
│  - Key Players (brands to cover in the article)                                                                 │
│    - Flyin’ Miata — specialist for Miata headers, cat‑backs, turbo solutions (strong Miata community cachet).   │
│    - Racing Beat — long history with Miata performance parts (headers, crosspipes, cat‑backs).                  │
│    - Goodwin Racing — UK specialist, quality Miata exha

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8e54210f-0965-4df8-924a-966495f6bae5                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Exhaust Systems on Mazda Miatas.              │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 216ac35d-fea3-4042-8078-819e04ed17e0                                                                     │
│  Agent: Content Writer                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Exhaust Systems for Mazda Miata (MX‑5): The Ultimate Buyer's Guide for NA, NB, NC & ND                       │
│                                                                                                                 │
│  Meta description: Find the best exhaust for your Miata — sound, power, fitment, and legal tips for             │
│  NA/NB/NC/ND. Brands, dyno expectations, and a practical buying checklist.                                      │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Choosing the right exhaust is one of the most impactful—and often most confusing—upgrades for any Miata        │
│  owner. The MX‑5’s light chassis and small‑displacement engines make exhaust choices matter for sound,          │
│  throttle response, weight, and modest power gains when combined with complementary mods. Whether you own a     │
│  classic NA or the modern ND, a carefully chosen cat‑back, header, or downpipe can transform the driving        │
│  experience without undermining comfort or legality.                                                            │
│                                                                                                                 │
│  This guide walks you step‑by‑step through exhaust basics, realistic performance expectations, emissions and    │
│  CARB considerations, and generation‑specific recommendations for NA, NB, NC, and ND Miatas. You’ll find brand  │
│  picks, cost ranges, installation advice, and a practical buying checklist so you can decide: sound, power, or  │
│  legal compliance.                                                                                              │
│                                                                                                                 │
│  ## Exhaust Basics: What Every Miata Owner Should Know                                                          │
│                                                                                                                 │
│  An exhaust system is several components working together: the exhaust manifold or header at the flange, the    │
│  mid‑pipe or downpipe (critical on turbo builds), catalytic converters (OEM, high‑flow, or catless), and the    │
│  cat‑back or axle‑back section with the muffler and resonators. Headers replace the restrictive OEM manifold    │
│  to improve scavenging; mid‑pipes and downpipes remove downstream bottlenecks and influence turbo spool on      │
│  boosted cars. Understanding these pieces helps you choose the right combination for your goals.                │
│                                                                                                                 │
│  Terminology matters when selecting parts: cat‑back replaces everything from the catalytic converter rearward,  │
│  while axle‑back replaces only the portion behind the rear axle. Mufflers and resonators shape tone and         │
│  control drone; Helmholtz chambers and internal baffling are deliberate design choices to quiet specific        │
│  frequencies without killing character. Materials range

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 55a857de-2855-4019-a53f-385d750b9a0d                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6d7e5555-3e99-41ca-b224-5cbea15ab28c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Exhaust Systems for Mazda Miata (MX‑5): The Ultimate Buyer's Guide for NA, NB, NC & ND         │
│                                                                                                                 │
│  Meta description: Find the best exhaust for your Miata — sound, power, fitment, and legal tips for             │
│  NA/NB/NC/ND. Brands, dyno expectations, and a practical buying checklist.                                      │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Choosing the right exhaust is one of the most impactful—and often most confusing—upgrades for any Miata        │
│  owner. The MX‑5’s light chassis and small‑displacement engines make exhaust choices matter for sound,          │
│  throttle response, weight, and modest power gains when combined with complementary mods. Whether you own a     │
│  classic NA or the modern ND, a carefully chosen cat‑back, header, or downpipe can transform the driving        │
│  experience without undermining comfort or legality.                                                            │
│                                                                                                                 │
│  This guide walks you step‑by‑step through exhaust basics, realistic performance expectations, emissions and    │
│  CARB considerations, and generation‑specific recommendations for NA, NB, NC, and ND Miatas. You’ll find brand  │
│  picks, cost ranges, installation advice, and a practical buying checklist so you can decide: sound, power, or  │
│  legal compliance.                                                                                              │
│                                                                                                                 │
│  ## Exhaust Basics: What Every Miata Owner Should Know                                                          │
│                                                                                                                 │
│  An exhaust system is several components working together: the exhaust manifold or header at the flange, the    │
│  mid‑pipe or downpipe (critical on turbo builds), catalytic converters (OEM, high‑flow, or catless), and the    │
│  cat‑back or axle‑back section with the muffler and resonators. Headers replace the restrictive OEM manifold    │
│  to improve scavenging; mid‑pipes and downpipes remove downstream bottlenecks and influence turbo spool on      │
│  boosted cars. Understanding these pieces helps you choose the right combination for your goals.                │
│                                                                                                                 │
│  Terminology matters when selecting parts: cat‑back replaces everything from the catalytic converter rearward,  │
│  while axle‑back replaces only the portion behind the rear axle. Mufflers and resonators shape tone and         │
│  control drone; Helmholtz chambers and internal baffli